# We will separate lowmass from massive systems

In [1]:
import numpy as np
import pandas as pd

In [2]:
# read list of all known planets (post processed exoplanet.eu data)
### isys      : index of system
### totpl_sys : number of planets in the system
### ipl_sys   : index of planet in the system

data_full=pd.read_csv('../planets_july2021.txt',delimiter='\s{3,}',header=None,engine='python',names=['isys','totpl_sys','ipl_sys','pl_name','m','m_est','r','P','a','e','inc','w','lambda','idetect','st_met','st_m','st_r','st_age','date'])
data_full

,isys,totpl_sys,ipl_sys,pl_name,m,m_est,r,P,a,e,inc,w,lambda,idetect,st_met,st_m,st_r,st_age,date
0,1,1,1,11 Com b,6167.6483,6167.6483,0.0000,3.260300e+02,1.2900,0.23100,0.000,94.80,0.0,1.0,-0.35,2.70,19.00,0.00,2008.0
1,2,1,1,11 Oph b,6676.3203,6676.3203,0.0000,7.300000e+05,243.0000,0.00000,0.000,0.00,0.0,2.0,0.00,0.02,0.00,0.01,2007.0
2,3,1,1,11 UMi b,3338.1601,3338.1601,0.0000,5.162200e+02,1.5400,0.08000,0.000,117.63,0.0,1.0,0.04,1.80,24.08,1.56,2009.0
3,4,1,1,14 And b,1694.5137,1694.5137,0.0000,1.858400e+02,0.8300,0.00000,0.000,0.00,0.0,1.0,-0.24,2.20,11.00,0.00,2008.0
4,5,2,1,14 Her b,1475.1489,1475.1489,0.0000,1.773400e+03,2.7700,0.36900,0.000,22.60,0.0,1.0,0.43,0.90,0.71,5.10,2002.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4712,3542,4,1,ups And b,197.1104,197.1104,0.0000,4.617100e+00,0.0590,0.01186,90.000,44.52,0.0,1.0,0.09,1.27,1.63,3.80,1996.0
4713,3542,4,2,ups And c,2893.0721,2893.0721,0.0000,2.409370e+02,0.8610,0.24450,11.347,247.63,0.0,1.0,0.09,1.27,1.63,3.80,1999.0
4714,3542,4,3,ups And d,7496.5539,7496.5539,0.0000,1.281439e+03,2.5500,0.31600,25.609,252.99,0.0,1.0,0.09,1.27,1.63,3.80,1999.0
4715,3542,4,4,ups And e,336.6773,336.6773,0.0000,3.848860e+03,5.2456,0.00536,0.000,7.30,0.0,1.0,0.09,1.27,1.63,3.80,2010.0


In [3]:
# returns planet's name when given (isys,ipl_sys)
def plname(df,isys,ipl_sys):
    plname = df[(df['isys']==isys)&(df['ipl_sys']==ipl_sys)]['pl_name'].values[0]
    return(plname)

In [4]:
# keep systems with at least 2 planets
data = data_full[data_full['totpl_sys']>1]

In [5]:
datag = data.groupby('isys') # datag can be divided into individual dataframes for each system
sysl = sorted(list(set(data['isys']))) # least of isys with at least 2 planets

.
### Low-mass systems data table

In [6]:
lowmass=[] # main lowmass table
for isys in sysl:
    i=1
    data_isys = datag.get_group(isys) # dataframe for system isys
    itotpl = data_isys['totpl_sys'].values[0] # number of planets in isys
    
    # lowmass condition
    m_l = np.array(data_isys['m'] > 20)
    if True in m_l:
        continue
    
    # loop per planetary tuple in isys
    counter=0
    for intervalo in range(itotpl-1):
        Pi0 = data_isys['P'].values[counter]
        Pi1 = data_isys['P'].values[counter+1]

        mi0 = data_isys['m'].values[counter]
        mi1 = data_isys['m'].values[counter+1]

        id0 = data_isys['ipl_sys'].values[counter]
        id1 = data_isys['ipl_sys'].values[counter+1]
        
        pl0=plname(data,isys,id0)
        pl1=plname(data,isys,id1)
        
        if Pi0!=0. and Pi1!=0.:
            p1p0_ = Pi1/Pi0
            lowmass.append([pl0,pl1,p1p0_,mi0,mi1,Pi0,Pi1])
        
        counter+=1
        
print('# of low-mass mean motion ratios: '+str(len(lowmass)))

# list to dataframe
lowmass = pd.DataFrame(np.matrix(lowmass),columns=['p1p0','pl_1','pl_2','m1','m2','P1','P2'])
# specify dtypes
lowmass = lowmass.astype(dtype={'p1p0':float,'pl_1':str,'pl_2':str,'m1':float,'m2':float,'P1':float,'P2':float})

lowmass.head(6) # show first 6 rows

# of low-mass mean motion ratios: 813


ValueError: could not convert string to float: 'AU Mic b'

.
### Massive systems data table

In [ ]:
massive=[] # main massive table
lmassive=[]
for isys in sysl:
    i=1
    data_isys = datag.get_group(isys) # dataframe for system isys
    itotpl = data_isys['totpl_sys'].values[0] # number of planets in isys
    
    # massive condition
    m_l = np.array(data_isys['m'] >= 100)
    if not True in m_l:
        continue
        
    # loop per planetary tuple in isys
    counter=0
    for intervalo in range(itotpl-1):
        Pi0 = data_isys['P'].values[counter]
        Pi1 = data_isys['P'].values[counter+1]

        mi0 = data_isys['m'].values[counter]
        mi1 = data_isys['m'].values[counter+1]
        mmax = max(mi0,mi1)
        
        id0 = data_isys['ipl_sys'].values[counter]
        id1 = data_isys['ipl_sys'].values[counter+1]

        pl0=plname(data,isys,id0)
        pl1=plname(data,isys,id1)
        
        # we only allow ratios between at least one massive planet
        if Pi0!=0. and Pi1!=0. and mmax<10.*317.92 and mmax>100.:
            p1p0_ = Pi1/Pi0
            massive.append([p1p0_,pl0,pl1,mi0,mi1,Pi0,Pi1])

        counter+=1
        
print('# of massive mean motion ratios: '+str(len(massive)))

# list to dataframe
massive = pd.DataFrame(np.matrix(massive),columns=['p1p0','pl_1','pl_2','m1','m2','P1','P2'])
# specify dtypes
massive = massive.astype(dtype={'p1p0':float,'pl_1':str,'pl_2':str,'m1':float,'m2':float,'P1':float,'P2':float})

massive.head(6) # show first 6 rows

.
### Save files

In [ ]:
lowmass.to_csv('hist_lowmass.dat',index=False,header=None)
massive.to_csv('hist_massive.dat',index=False,header=None)